In [90]:
import pandas as pd

import sklearn
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
from sklearn.metrics import log_loss, roc_auc_score, classification_report
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler

from xgboost import XGBClassifier

In [44]:
df = pd.read_csv("../Data/Clean_Data.csv")

In [8]:
df.columns

Index(['row ID', 'BATHRM', 'HF_BATHRM', 'HEAT', 'HEAT_D', 'AC', 'NUM_UNITS',
       'ROOMS', 'BEDRM', 'AYB', 'YR_RMDL', 'EYB', 'STORIES', 'SALEDATE',
       'PRICE', 'SALE_NUM', 'GBA', 'BLDG_NUM', 'STYLE', 'STYLE_D', 'STRUCT',
       'STRUCT_D', 'GRADE', 'GRADE_D', 'CNDTN', 'CNDTN_D', 'EXTWALL',
       'EXTWALL_D', 'ROOF', 'ROOF_D', 'INTWALL', 'INTWALL_D', 'KITCHENS',
       'FIREPLACES', 'USECODE', 'LANDAREA', 'GIS_LAST_MOD_DTTM', 'QUALIFIED'],
      dtype='object')

Firstly we want to create a base version

This won't have any pre-processing done to it. The way the Kaggle system works means we can use this to get a (vague) idea of whether we improved upon our model through data engineering.

In [45]:
feats = ['BATHRM', 'HF_BATHRM', 'HEAT', 'HEAT_D', 'AC', 'NUM_UNITS',
       'ROOMS', 'BEDRM', 'AYB', 'EYB', 'YR_RMDL', 'STORIES', 'SALEDATE',
       'PRICE', 'SALE_NUM', 'GBA', 'STYLE', 'STYLE_D', 'STRUCT',
       'STRUCT_D', 'GRADE', 'GRADE_D', 'CNDTN', 'CNDTN_D', 'EXTWALL',
       'EXTWALL_D', 'ROOF', 'ROOF_D', 'INTWALL', 'INTWALL_D', 'KITCHENS',
       'FIREPLACES', 'USECODE', 'LANDAREA']
#We have removed ROW_ID, BLDG_NUM, GIS_LAST_MOD_DTTM, QUALIFIED, 

x = df[feats]
y = df['QUALIFIED']

In [46]:
#Label Encoding for the String columns
le = preprocessing.LabelEncoder()

for column_name in x.columns:
    if x[column_name].dtype == object:
        x[column_name].fillna('MISSING_DATA', inplace=True)
        x[column_name] = le.fit_transform(x[column_name])
    else:
        x[column_name].fillna(-1, inplace=True)

c:\users\j_m_g\appdata\local\programs\python\python38\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
<ipython-input-46-7e7e17473232>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[column_name] = le.fit_transform(x[column_name])


In [47]:
x.head()

,BATHRM,HF_BATHRM,HEAT,HEAT_D,AC,NUM_UNITS,ROOMS,BEDRM,AYB,EYB,...,EXTWALL,EXTWALL_D,ROOF,ROOF_D,INTWALL,INTWALL_D,KITCHENS,FIREPLACES,USECODE,LANDAREA
0,3.0,1.0,7.0,12,2,2.0,11.0,5.0,1,1972,...,14.0,6,2.0,0,6.0,3,2.0,4.0,24,1680
1,3.0,1.0,13.0,7,2,2.0,9.0,5.0,1,1984,...,14.0,6,2.0,0,6.0,3,2.0,4.0,24,1680
2,2.0,1.0,7.0,12,2,1.0,11.0,3.0,1,1985,...,14.0,6,13.0,9,6.0,3,1.0,0.0,13,2032
3,3.0,2.0,13.0,7,2,1.0,10.0,5.0,1,1972,...,14.0,6,2.0,0,6.0,3,1.0,4.0,11,2196
4,3.0,1.0,7.0,12,2,2.0,7.0,3.0,1,1967,...,14.0,6,2.0,0,6.0,3,2.0,1.0,24,1424


In [23]:
x.describe()

,BATHRM,HF_BATHRM,HEAT,HEAT_D,AC,NUM_UNITS,ROOMS,BEDRM,AYB,EYB,...,EXTWALL,EXTWALL_D,ROOF,ROOF_D,INTWALL,INTWALL_D,KITCHENS,FIREPLACES,USECODE,LANDAREA
count,74987.000000,74987.000000,74987.000000,74987.000000,74987.000000,74987.000000,74987.000000,74987.000000,74987.000000,74987.000000,...,74987.000000,74987.000000,74987.000000,74987.000000,74987.000000,74987.000000,74987.000000,74987.000000,74987.000000,74987.000000
mean,2.035660,0.608292,7.645952,7.754091,1.620921,1.195967,7.356168,3.372758,1933.740968,1966.257591,...,13.355608,8.147799,3.992239,4.363570,6.144772,3.627389,1.217491,0.619761,13.124182,3419.506875
std,1.067649,0.617184,5.038308,2.751754,0.486259,0.593972,2.356113,1.169626,36.083904,16.418251,...,3.938126,5.919490,3.351630,4.238063,1.960660,2.235960,0.624920,0.889512,3.971845,5520.064430
min,0.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,-1.000000,1900.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,11.000000,0.000000
25%,1.000000,0.000000,1.000000,5.000000,1.000000,1.000000,6.000000,3.000000,1914.000000,1955.000000,...,14.000000,6.000000,1.000000,0.000000,6.000000,3.000000,1.000000,0.000000,11.000000,1600.000000
50%,2.000000,1.000000,7.000000,7.000000,2.000000,1.000000,7.000000,3.000000,1930.000000,1964.000000,...,14.000000,6.000000,2.000000,2.000000,6.000000,3.000000,1.000000,0.000000,12.000000,2370.000000
75%,3.000000,1.000000,13.000000,12.000000,2.000000,1.000000,8.000000,4.000000,1947.000000,1970.000000,...,14.000000,6.000000,6.000000,8.000000,6.000000,3.000000,1.000000,1.000000,13.000000,4200.000000
max,24.000000,11.000000,13.000000,13.000000,2.000000,6.000000,101.000000,54.000000,2018.000000,2018.000000,...,24.000000,24.000000,15.000000,15.000000,11.000000,11.000000,44.000000,13.000000,81.000000,691817.000000


In [24]:
#Split into test and train datasets
seed = 42

x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.3, random_state = seed)

In [25]:
print(len(x))
print(len(x_train))
print(len(x_val))

74987
52490
22497


Lets make some predictions to upload

In [48]:
pred_df = pd.read_csv("../Data/Clean_Pred_Data.csv")

In [49]:
p = pred_df[feats]

#Label Encoding for the String columns
le = preprocessing.LabelEncoder()

for column_name in x.columns:
    if p[column_name].dtype == object:
        p[column_name].fillna('MISSING_DATA', inplace=True)
        p[column_name] = le.fit_transform(p[column_name])
    else:
        p[column_name].fillna(-1, inplace=True)

c:\users\j_m_g\appdata\local\programs\python\python38\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
<ipython-input-49-585002b8590c>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  p[column_name] = le.fit_transform(p[column_name])


In [111]:
#Create the ensemble model using the Max Voting sklearn model.
#First submission - Scored 0.85777
model1 = LogisticRegression(multi_class='multinomial', random_state=1)
model2 = DecisionTreeClassifier(random_state=1)
model = VotingClassifier(estimators=[('lr', model1), ('dt', model2)], voting='soft')
model.fit(x_train,y_train)
print(model.score(x_val,y_val))

y_pred = model.predict(x_val)
print(roc_auc_score(y_val, y_pred))

p_pred = model.predict(p)

upload_df = pd.DataFrame()
upload_df['row ID'] = pred_df['row ID']
upload_df['Predict-Qualified'] = p_pred
upload_df.to_csv(r'../Data/Ensemble_Prediction.csv', index = False)

0.8717948717948718
0.8689649279912601


In [117]:
#Create the ensemble model using the Max Voting sklearn model.
#Second & Third submission - Scored 0.89037
model1 = LogisticRegression(multi_class='multinomial', random_state=1)
model2 = DecisionTreeClassifier(random_state=1)
model3 = XGBClassifier(seed=seed)
model = VotingClassifier(estimators=[('lr', model1), ('dt', model2), ('xgb', model3)], voting='soft')
model.fit(x_train,y_train)
print(model.score(x_val,y_val))

y_pred = model.predict(x_val)
print(roc_auc_score(y_val, y_pred))

p_pred = model.predict(p)

upload_df = pd.DataFrame()
upload_df['row ID'] = pred_df['row ID']
upload_df['Predict-Qualified'] = p_pred
upload_df.to_csv(r'../Data/Ensemble_Prediction_Second.csv', index = False)

0.8939252544105231
0.8951497639186483


In [28]:
#Create the ensemble model using the Max Voting sklearn model.
#Fourth Submission - Scored 0.90499
model1 = LogisticRegression(multi_class='auto', random_state=seed)
model2 = DecisionTreeClassifier(random_state=seed)
model3 = XGBClassifier(seed=seed)
model4 = RandomForestClassifier(random_state=seed, criterion='entropy')
model5 = GradientBoostingClassifier(random_state=seed)
model6 = RandomForestClassifier(random_state=seed, criterion='gini')

model = VotingClassifier(estimators=[('lr', model1), ('dt', model2), ('xgb', model3), ('rfc1', model4), ('gbc', model5), ('rfc2', model6)], voting='soft')
model.fit(x_train,y_train)
print(model.score(x_val,y_val))

y_pred = model.predict(x_val)
print(roc_auc_score(y_val, y_pred))

p_pred = model.predict(p)

upload_df = pd.DataFrame()
upload_df['row ID'] = pred_df['row ID']
upload_df['Predict-Qualified'] = p_pred
upload_df.to_csv(r'../Data/Ensemble_Prediction_Fifth.csv', index = False)

c:\users\j_m_g\appdata\local\programs\python\python38\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9116771125038894
0.9165478009677234


In [31]:
#Create the ensemble model using the Max Voting sklearn model.
#Fourth Submission - Scored 0.90624
model1 = KNeighborsClassifier()
model2 = DecisionTreeClassifier(random_state=seed)
model3 = XGBClassifier(seed=seed)
model4 = RandomForestClassifier(random_state=seed, criterion='entropy')
model5 = GradientBoostingClassifier(random_state=seed)
model6 = RandomForestClassifier(random_state=seed, criterion='gini')

model = VotingClassifier(estimators=[('lr', model1), ('dt', model2), ('xgb', model3), ('rfc1', model4), ('gbc', model5), ('rfc2', model6)], voting='soft')
model.fit(x_train,y_train)
print(model.score(x_val,y_val))

y_pred = model.predict(x_val)
print(roc_auc_score(y_val, y_pred))

p_pred = model.predict(p)

upload_df = pd.DataFrame()
upload_df['row ID'] = pred_df['row ID']
upload_df['Predict-Qualified'] = p_pred
upload_df.to_csv(r'../Data/Ensemble_Prediction_Clean_Data.csv', index = False)

0.9129661732675468
0.9201440385588527


In [50]:
#Create a stacking model
model1 = KNeighborsClassifier()
model2 = DecisionTreeClassifier(random_state=seed)
model3 = XGBClassifier(seed=seed)
model4 = RandomForestClassifier(random_state=seed, criterion='entropy')
model5 = GradientBoostingClassifier(random_state=seed)
model6 = RandomForestClassifier(random_state=seed, criterion='gini')

estimators = [('lr', model1), ('dt', model2), ('xgb', model3), ('rfc1', model4), ('gbc', model5), ('rfc2', model6)]

stacking = StackingClassifier(
    estimators=estimators, final_estimator=LogisticRegression()
)

stacking.fit(x_train,y_train)
print(stacking.score(x_val,y_val))

y_pred = stacking.predict(x_val)
print(roc_auc_score(y_val, y_pred))

0.9170556074143219
0.9220165382066988


In [55]:
#Create a stacking model
model1 = KNeighborsClassifier()
model2 = DecisionTreeClassifier(random_state=seed)
model3 = XGBClassifier(seed=seed)
model4 = SVC()
model5 = GaussianNB()
model6 = RandomForestClassifier(random_state=seed, criterion='gini')

estimators = [('lr', model1), ('dt', model2), ('xgb', model3), ('rfc1', model4), ('gbc', model5), ('rfc2', model6)]

stacking = StackingClassifier(
    estimators=estimators, final_estimator=LogisticRegression()
)

stacking.fit(x_train,y_train)
print(stacking.score(x_val,y_val))

y_pred = stacking.predict(x_val)
print(roc_auc_score(y_val, y_pred))

0.9172334088989643
0.9212683631393127


In [72]:
model1 = KNeighborsClassifier()
model1.fit(x_train,y_train)
print(model1.score(x_val,y_val))

y_pred = model1.predict(x_val)
print(roc_auc_score(y_val, y_pred))

0.8940747655242921
0.9002342529869186


In [88]:
model2 = DecisionTreeClassifier(random_state=seed)
model2.fit(x_train,y_train)
print(model2.score(x_val,y_val))

y_pred = model2.predict(x_val)
print(roc_auc_score(y_val, y_pred))

print(classification_report(y_val, y_pred))

0.8768724718851403
0.8734219712231776
              precision    recall  f1-score   support

           0       0.89      0.90      0.89     12940
           1       0.86      0.85      0.85      9557

    accuracy                           0.88     22497
   macro avg       0.87      0.87      0.87     22497
weighted avg       0.88      0.88      0.88     22497



In [87]:
model3 = XGBClassifier(seed=seed, nthread=3)
model3.fit(x_train,y_train)
print(model3.score(x_val,y_val))

y_pred = model3.predict(x_val)
print(roc_auc_score(y_val, y_pred))

print(classification_report(y_val, y_pred))

0.917277859270125
0.922715816061089
              precision    recall  f1-score   support

           0       0.97      0.89      0.92     12940
           1       0.86      0.96      0.91      9557

    accuracy                           0.92     22497
   macro avg       0.91      0.92      0.92     22497
weighted avg       0.92      0.92      0.92     22497



In [39]:
model4 = SVC()
model4.fit(x_train,y_train)
print(model4.score(x_val,y_val))

y_pred = model4.predict(x_val)
print(roc_auc_score(y_val, y_pred))

0.9079877316975596
0.914899960038031


In [40]:
model5 = GaussianNB()
model5.fit(x_train,y_train)
print(model5.score(x_val,y_val))

y_pred = model5.predict(x_val)
print(roc_auc_score(y_val, y_pred))

0.9077654798417567
0.9173739309849842


In [41]:
model6 = RandomForestClassifier(random_state=seed, criterion='gini')
model6.fit(x_train,y_train)
print(model6.score(x_val,y_val))

y_pred = model6.predict(x_val)
print(roc_auc_score(y_val, y_pred))

0.9083433346668445
0.9149081675245849


In [35]:
p_pred = stacking.predict(p)

upload_df = pd.DataFrame()
upload_df['row ID'] = pred_df['row ID']
upload_df['Predict-Qualified'] = p_pred
upload_df.to_csv(r'../Data/Ensemble_Prediction_Stacking.csv', index = False)

In [64]:
KNNModel = KNeighborsClassifier(n_jobs = 3)
KNNModel.fit(x_train,y_train)
print(KNNModel.score(x_val,y_val))

y_pred = KNNModel.predict(x_val)
print(roc_auc_score(y_val, y_pred))

print(classification_report(y_val, y_pred))

0.8940747655242921
0.9002342529869186
              precision    recall  f1-score   support

           0       0.95      0.86      0.90     12940
           1       0.83      0.94      0.88      9557

    accuracy                           0.89     22497
   macro avg       0.89      0.90      0.89     22497
weighted avg       0.90      0.89      0.89     22497



In [71]:
KNNModel = KNeighborsClassifier(n_neighbors = 20, leaf_size = 20, n_jobs = 3, p=1)
KNNModel.fit(x_train,y_train)
print(KNNModel.score(x_val,y_val))

y_pred = KNNModel.predict(x_val)
print(roc_auc_score(y_val, y_pred))

print(classification_report(y_val, y_pred))

0.8980308485575854
0.907243106883793
              precision    recall  f1-score   support

           0       0.97      0.85      0.91     12940
           1       0.82      0.97      0.89      9557

    accuracy                           0.90     22497
   macro avg       0.90      0.91      0.90     22497
weighted avg       0.91      0.90      0.90     22497



In [74]:
#List Hyperparameters that we want to tune.
leaf_size = list([10,15,20,25])
n_neighbors = list([10,15,20,25])
p=[1,2]

#Convert to dictionary
hyperparameters = dict(leaf_size=leaf_size, n_neighbors=n_neighbors, p=p)

#Create new KNN object
knn_2 = KNeighborsClassifier(n_jobs = 3)

#Use GridSearch
clf = GridSearchCV(knn_2, hyperparameters, cv=10)

#Fit the model
best_model = clf.fit(x_train,y_train)
#Print The value of best Hyperparameters
print('Best leaf_size:', best_model.best_estimator_.get_params()['leaf_size'])
print('Best p:', best_model.best_estimator_.get_params()['p'])
print('Best n_neighbors:', best_model.best_estimator_.get_params()['n_neighbors'])

Best leaf_size: 10
Best p: 1
Best n_neighbors: 25


In [83]:
#List Hyperparameters that we want to tune.
leaf_size = list([3,5,7])
n_neighbors = list([20,25,30])
p=[1,2]

#Convert to dictionary
hyperparameters = dict(leaf_size=leaf_size, n_neighbors=n_neighbors, p=p)

#Create new KNN object
knn_2 = KNeighborsClassifier(n_jobs = 3)

#Use GridSearch
clf = GridSearchCV(knn_2, hyperparameters, cv=10)

#Fit the model
best_model = clf.fit(x_train,y_train)
#Print The value of best Hyperparameters
print('Best leaf_size:', best_model.best_estimator_.get_params()['leaf_size'])
print('Best p:', best_model.best_estimator_.get_params()['p'])
print('Best n_neighbors:', best_model.best_estimator_.get_params()['n_neighbors'])

Best leaf_size: 3
Best p: 1
Best n_neighbors: 25


In [84]:
y_pred = best_model.best_estimator_.predict(x_val)
print(roc_auc_score(y_val, y_pred))

print(classification_report(y_val, y_pred))

0.90849120683044
              precision    recall  f1-score   support

           0       0.98      0.84      0.91     12940
           1       0.82      0.98      0.89      9557

    accuracy                           0.90     22497
   macro avg       0.90      0.91      0.90     22497
weighted avg       0.91      0.90      0.90     22497



Using the best model, we'll scale our data and make new predictions

In [91]:
scaler = StandardScaler()

scaler.fit(x)
x_scaler = scaler.transform(x)

#Split into test and train datasets
seed = 42

x_train, x_val, y_train, y_val = train_test_split(x_scaler, y, test_size=0.2, random_state = seed)

In [92]:
pred_df = pd.read_csv("../Data/Clean_Pred_Data.csv")

p = pred_df[feats]

#Label Encoding for the String columns
le = preprocessing.LabelEncoder()

for column_name in x.columns:
    if p[column_name].dtype == object:
        p[column_name].fillna('MISSING_DATA', inplace=True)
        p[column_name] = le.fit_transform(p[column_name])
    else:
        p[column_name].fillna(-1, inplace=True)
        
scaler = StandardScaler()

scaler.fit(p)
p = scaler.transform(p)

c:\users\j_m_g\appdata\local\programs\python\python38\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
<ipython-input-92-73144384f05c>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  p[column_name] = le.fit_transform(p[column_name])


In [93]:
#Create a stacking model
model1 = KNeighborsClassifier()
model2 = DecisionTreeClassifier(random_state=seed)
model3 = XGBClassifier(seed=seed)
model4 = SVC()
model5 = GaussianNB()
model6 = RandomForestClassifier(random_state=seed, criterion='gini')

estimators = [('lr', model1), ('dt', model2), ('xgb', model3), ('rfc1', model4), ('gbc', model5), ('rfc2', model6)]

stacking = StackingClassifier(
    estimators=estimators, final_estimator=LogisticRegression()
)

stacking.fit(x_train,y_train)
print(stacking.score(x_val,y_val))

y_pred = stacking.predict(x_val)
print(roc_auc_score(y_val, y_pred))

0.9156554207227631
0.9195503450915733


In [94]:
model1 = KNeighborsClassifier()
model2 = DecisionTreeClassifier(random_state=seed)
model3 = XGBClassifier(seed=seed)
model4 = RandomForestClassifier(random_state=seed, criterion='entropy')
model5 = GradientBoostingClassifier(random_state=seed)
model6 = RandomForestClassifier(random_state=seed, criterion='gini')

model = VotingClassifier(estimators=[('lr', model1), ('dt', model2), ('xgb', model3), ('rfc1', model4), ('gbc', model5), ('rfc2', model6)], voting='soft')
model.fit(x_train,y_train)
print(model.score(x_val,y_val))

y_pred = model.predict(x_val)
print(roc_auc_score(y_val, y_pred))

p_pred = model.predict(p)

upload_df = pd.DataFrame()
upload_df['row ID'] = pred_df['row ID']
upload_df['Predict-Qualified'] = p_pred
upload_df.to_csv(r'../Data/Ensemble_Prediction_Clean_Data.csv', index = False)

0.9109881317509001
0.9167658391955462
